In [ ]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

In [ ]:
# Importing the dataset
dataset = pd.read_csv('Ads_Optimisation.csv')

## MAB Formulation
To maximize the total number of clicks we receive, each time a user visits the webpage, the MAB agent must decide which ad to display to the user based on the available data of ad click history. 

The goal of the MAB agent is to balance exploration (trying out different ads to gather more data) and exploitation (using the best performing ads to maximize clicks) to achieve the maximum total number of clicks over a certain period of time.

Therefore, the MAB agent must continually update its knowledge of which ad is most likely to be clicked by a user and make decisions based on that knowledge to optimize the number of clicks received.

## Random Selection

In [ ]:
# Implementing Random Selection
import random
N = 10000
d = 10
ads_selected = []
total_reward = 0
for n in range(0, N):
    ad = random.randrange(d)
    ads_selected.append(ad)
    reward = dataset.values[n, ad]
    total_reward = total_reward + reward

In [ ]:
pd.Series(ads_selected).tail(1000).value_counts(normalize=True)

4    0.115
0    0.110
7    0.108
9    0.104
3    0.101
2    0.098
6    0.097
5    0.094
8    0.090
1    0.083
dtype: float64

## Epsilon-Greedy

In [ ]:
np.seterr(invalid='ignore')
ads = np.loadtxt('Ads_Optimisation.csv', delimiter=',', skiprows=1)

# define the number of arms (ads) and the total number of time steps
num_arms = ads.shape[1]
num_steps = 2000

# initialize the number of times each arm was pulled and the total rewards obtained
num_pulls = np.zeros(num_arms)
total_rewards = np.zeros(num_arms)

# define the epsilon-greedy action function
def epsilon_greedy_action(epsilon):
    if np.random.uniform() < epsilon:
        # explore: choose a random arm
        return np.random.randint(num_arms)
    else:
        # exploit: choose the arm with the highest estimated reward
        return np.argmax(total_rewards / num_pulls)

# un the bandit algorithm for each value of epsilon
for epsilon in [0.01, 0.3]:

    # reset the number of times each arm was pulled and the total rewards obtained
    num_pulls = np.zeros(num_arms)
    total_rewards = np.zeros(num_arms)

    # run the bandit algorithm for the specified number of time steps
    for t in range(num_steps):
        # choose which arm to pull using the epsilon-greedy action
        arm = epsilon_greedy_action(epsilon)
        
        # update the number of times the chosen arm was pulled and the total rewards obtained
        reward = ads[t][arm]
        num_pulls[arm] += 1
        total_rewards[arm] += reward

    # compute the total rewards obtained
    print("Total rewards obtained with epsilon = {}: {}".format(epsilon, np.sum(total_rewards)))


Total rewards obtained with epsilon = 0.01: 337.0
Total rewards obtained with epsilon = 0.3: 405.0


Thus, choosing a higher value of epsilon can lead to a higher total reward because the algorithm explores more frequently, which can result in a better understanding of the expected reward of each arm. However, this approach may not be optimal in the long run, as it can lead to suboptimal decisions when the algorithm becomes too focused on exploration and does not exploit the highest-reward arm enough.



## UCB Algorithm

In [ ]:
# initialize the number of times each arm was pulled and the total rewards obtained
num_pulls = np.zeros(num_arms)
total_rewards = np.zeros(num_arms)

# run the UCB algorithm with the specified value of c
c = 1.5
for t in range(num_steps):
    # Choose which arm to pull using the UCB action
    if t < num_arms:
        arm = t  # Choose each arm once in the first num_arms steps
    else:
        ucb_values = total_rewards / num_pulls + c * np.sqrt(np.log(t) / num_pulls)
        arm = np.argmax(ucb_values)
        
    # Update the number of times the chosen arm was pulled and the total rewards obtained
    reward = ads[t][arm]
    num_pulls[arm] += 1
    total_rewards[arm] += reward

# Compute the total rewards obtained
print("Total rewards obtained with UCB algorithm (c = 1.5): {}".format(np.sum(total_rewards)))

Total rewards obtained with UCB algorithm (c = 1.5): 319.0


In UCB, the algorithm tends to explore less often as it becomes more confident about the rewards of each arm. If the UCB algorithm becomes too confident too quickly, it may miss out on better rewards that could be obtained by exploring more.

On the other hand, the epsilon-greedy algorithm always explores with a fixed probability, regardless of how confident it is about the rewards of each arm. This can lead to more exploration and potentially better rewards.

## Action Value vs Optimal Action

- **Epsilon-greedy approach:** The action value estimated is the sample average of the rewards obtained for each action. The optimal action is the action that has the highest estimated action value. However, since the epsilon-greedy approach explores with a fixed probability, it may sometimes choose a suboptimal action, even if it has a lower estimated action value.

- **UCB approach:** The action value estimated is based on both the sample average of the rewards obtained for each action and the uncertainty or confidence in that estimate. The UCB approach tries to balance the exploration and exploitation trade-off by choosing actions that have high estimated action values as well as high uncertainty. The optimal action is the action that has the highest estimated action value with the highest uncertainty or confidence.